In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, accuracy_score, roc_auc_score, roc_curve,
    confusion_matrix, precision_score, recall_score, f1_score, log_loss, balanced_accuracy_score
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
X_train = pd.read_csv('/content/X_Train_Data_Input.csv')
X_test = pd.read_csv('/content/X_Test_Data_Input.csv')    # Unseen Data
Y_train = pd.read_csv('/content/Y_Train_Data_Target.csv')
Y_test = pd.read_csv('/content/Y_Test_Data_Target.csv')

# Preprocessing
# Drop the ID column from both X_train and X_test
X_train_cleaned = X_train.drop(columns=["ID"])
X_test_cleaned = X_test.drop(columns=["ID"])

# Handle missing values using median imputation
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train_cleaned)
X_test_imputed = imputer.transform(X_test_cleaned)

# Standardize the features (scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Build a simple neural network using Keras
from tensorflow.keras.layers import Dropout, LeakyReLU

# Build a more complex model with dropout to prevent overfitting
model = Sequential([
    Dense(128, input_shape=(X_train_scaled.shape[1],)),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')  # Save in `.keras` format
]

# Train with the same callbacks
history = model.fit(X_train_scaled, Y_train['target'],
                    epochs=25, batch_size=32,
                    validation_data=(X_test_scaled, Y_test['target']),
                    callbacks=callbacks, verbose=2)


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks: early stopping and model checkpointing


# Train the model
history = model.fit(
    X_train_scaled, Y_train['target'],
    epochs=25, batch_size=32,
    validation_data=(X_test_scaled, Y_test['target']),  # Validate on test set
    callbacks=callbacks, verbose=2
)

# Make predictions on the test set
Y_pred_proba = model.predict(X_test_scaled).flatten()  # Predicted probabilities
Y_pred = (Y_pred_proba > 0.6).astype(int)  # Convert probabilities to binary predictions

# Evaluate the predictions against the actual target values in Y_test
print(classification_report(Y_test['target'], Y_pred))

# Calculate metrics
accuracy = accuracy_score(Y_test['target'], Y_pred)
auc_roc = roc_auc_score(Y_test['target'], Y_pred_proba)
precision = precision_score(Y_test['target'], Y_pred)
recall = recall_score(Y_test['target'], Y_pred)
f1 = f1_score(Y_test['target'], Y_pred)
balanced_acc = balanced_accuracy_score(Y_test['target'], Y_pred)
logloss = log_loss(Y_test['target'], Y_pred_proba)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"AUC-ROC: {auc_roc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Balanced Accuracy: {balanced_acc:.4f}")
print(f"Log Loss: {logloss:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(Y_test['target'], Y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted Negative', 'Predicted Positive'],
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Plot the ROC curve
fpr, tpr, _ = roc_curve(Y_test['target'], Y_pred_proba)
plt.figure()
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (area = {auc_roc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Display predicted targets for the unseen data
print("Predicted targets for the unseen data:")
print(Y_pred)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/25
24536/24536 - 62s - 3ms/step - accuracy: 0.9713 - loss: 0.0640 - val_accuracy: 0.9729 - val_loss: 0.0597
Epoch 2/25
24536/24536 - 73s - 3ms/step - accuracy: 0.9732 - loss: 0.0601 - val_accuracy: 0.9748 - val_loss: 0.0579
Epoch 3/25
24536/24536 - 69s - 3ms/step - accuracy: 0.9736 - loss: 0.0601 - val_accuracy: 0.9752 - val_loss: 0.0573
Epoch 4/25
24536/24536 - 86s - 3ms/step - accuracy: 0.9738 - loss: 0.0592 - val_accuracy: 0.9755 - val_loss: 0.0573
Epoch 5/25
24536/24536 - 59s - 2ms/step - accuracy: 0.9740 - loss: 0.0592 - val_accuracy: 0.9753 - val_loss: 0.0564
Epoch 6/25
24536/24536 - 60s - 2ms/step - accuracy: 0.9741 - loss: 0.0590 - val_accuracy: 0.9754 - val_loss: 0.0563
Epoch 7/25
24536/24536 - 61s - 2ms/step - accuracy: 0.9742 - loss: 0.0591 - val_accuracy: 0.9752 - val_loss: 0.0569
Epoch 8/25
24536/24536 - 61s - 2ms/step - accuracy: 0.9744 - loss: 0.0588 - val_accuracy: 0.9749 - val_loss: 0.0572
Epoch 9/25
24536/24536 - 81s - 3ms/step - accuracy: 0.9743 - loss: 0.059